In [1]:
import numpy as np              # Python's standard numerical library
import matplotlib.pyplot as plt # Python's scientific visualization library
from matplotlib.widgets import Cursor
import matplotlib.colors as mcolors
import matplotlib.animation as animation
from matplotlib.path import Path
from scipy.stats import norm
from ipywidgets import interact # interactive plots in Jupyter Notebook
import pyidi                    # Python HSC data analysis library
# from scipy.interpolate import make_interp_spline, BSpline
import pickle as pk
import os
from pixel_setter import PixelSetter, play_video, detect_peaks
from skimage import feature, measure
# from skimage.morphology import skeletonize
from skimage.feature import match_template
from mpl_toolkits import mplot3d
from skimage.transform import resize
from scipy.signal import correlate2d
%matplotlib qt

c:\Users\thijs\anaconda3\lib\site-packages\skimage\util\dtype.py:27: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


Open the file to analyze and set an analysis method (Lucas-Kanade implementation)

In [2]:
# file_name = 'Single_line_15_S01'
level_of_eccentricity   = 7     # level of eccentricity (0-7)
impact_location         = 0
camera_angle            = 0     # camera angle (0/1)
Tension                 = 30    # Tuned Tension (0.0 - 0.99 N)
impact_location         = 0     # Impact location (0 - 9)
has_mass                = 0     # has mass (0/1)
version                 = 0     # version (0/1)
# file_name = f"Full_web_ecc{str(level_of_eccentricity)}_Floc{str(impact_location)}_ca{str(camera_angle)}_T{str(Tension)}_m{str(has_mass)}_v{version}"

folder_paths = ['H:/My Drive/PHD/HSC', 'D:/HSC', 'F:/', 'E:/thijs/', 'C:/Users/thijs/Documents/HSC/']

file_name = "Full_web_ecc0_new_Floc4_v1"
file_name_video = file_name + "_S01.cihx"

# Search for the file in the folders and their subdirectories
for folder_path in folder_paths:
    for root, dirs, files in os.walk(folder_path, topdown=False):
        if file_name_video in files:
            file_path = os.path.join(root, file_name_video)
            print(file_path)
            break

# video_path = path + file_name_video +'/' + file_name_video+'.cihx'
video = pyidi.pyIDI(file_path)
# video_path = "C:/Users/thijs/Documents/GitHub/pyidi/data/"
# file_name = 'synthetic_data_thijs_without_rotation'
# video = pyidi.pyIDI(video_path + file_name +'.cih')
# video.set_method('sof')
mraw = video.mraw
video.info

# ani = play_video(video, range(400,1200), interval=30)
# writer = animation.FFMpegWriter(fps=60)
# ani.save('anim/'+file_name + '_anim.mp4', writer=writer)
# print('animation generated')

# data_dir = r'C:\Users\thijs\Documents\GitHub\LDAQ\examples\Data'
# measurement_path = os.path.join(data_dir, file_name+'.pkl')
# with open(measurement_path, 'rb') as f:
#     measurement = pk.load(f)


# file_name = 'Full_web12_S01'

H:/My Drive/PHD/HSC\ecc0\web0\Full_web_ecc0_new_Floc4_v1_S01\Full_web_ecc0_new_Floc4_v1_S01.cihx


c:\Users\thijs\anaconda3\lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 10000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


{'Date': '2023/9/16',
 'Camera Type': 'FASTCAM SA-Z type 2100K-M-64GB',
 'Record Rate(fps)': 8000.0,
 'Shutter Speed(s)': 8104.0,
 'Total Frame': 10000,
 'Original Total Frame': 87371,
 'Image Width': 1024,
 'Image Height': 512,
 'File Format': 'Mraw',
 'EffectiveBit Depth': 12,
 'EffectiveBit Side': 'Higher',
 'Color Bit': 16,
 'Comment Text': ''}

Plot a frame in the video:

In [3]:
%matplotlib qt
sequential_image_n = 0
still_image = mraw[sequential_image_n]

sigma_vec = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])*3
fig, ax = plt.subplots(ncols=3, nrows=3, figsize=(10, 5))
for ax_i, sigma in enumerate(sigma_vec):
    row = ax_i // 3
    col = ax_i % 3
    edges = feature.canny(still_image, sigma=sigma)
    ax[row, col].imshow((2**16-1)-edges, cmap='gray')
    # skeleton = skeletonize(edges)
    # contours = measure.find_contours(edges, sigma)
    # ax[row, col].imshow(still_image, cmap='gray')
    # for contour in contours:
        # ax[row, col].plot(contour[:, 1], contour[:, 0], linewidth=2)


In [4]:
pix_set = PixelSetter(still_image)
pix_set.get_tracking_pixels()
pix_set.choose_reference_centers()
# pix_set, (fig, ax) = PixelSetter.load(file_name)

Axes(0,0;1x1)


In [ ]:
sigma_vec = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])*3
fig, ax = plt.subplots(ncols=3, nrows=3, figsize=(10, 5))
for ax_i, sigma in enumerate(sigma_vec):
    row = ax_i // 3
    col = ax_i % 3
    contours = measure.find_contours(still_image, sigma)
    ax[row, col].imshow(still_image, cmap='gray')
    for contour in contours:
        ax[row, col].plot(contour[:, 1], contour[:, 0], linewidth=2)

In [ ]:
%matplotlib qt
ani = play_video(video, range(400,1200), interval=30)

Find the gradient off the intensity accross the still figure

In [ ]:
gy, gx = np.gradient(still_image, edge_order=2)
g_norm = np.sqrt(gx**2 + gy**2)
arg_g  = g_norm > 15000
pix_gi, pix_gj = np.where(arg_g)
fig, ax = plt.subplots()
ax.imshow(still_image, cmap='gray')
ax.plot(pix_gj, pix_gi, 'r.')

In [ ]:
def generate_Zi(pix_i_n, pix_j_n, mu, sigma, bits, theta):
    dist = norm(mu, sigma)
    slope = np.tan(theta)
    x = np.linspace(-pix_i_n, pix_i_n, pix_i_n*12)
    y = np.linspace(-pix_i_n, pix_i_n, pix_j_n*12)

    X, Y = np.meshgrid(x, y)
    Z = np.maximum(dist.pdf(Y+slope*X), dist.pdf(-X+slope*Y))
    Z = (1- Z / np.max(Z)) * (2**bits-1)

    compressed_Z = resize(Z, (pix_i_n, pix_j_n)).astype(np.uint16)
    return X, Y, Z, compressed_Z

bits = 16
pix_i_n = 7
pix_j_n = 7
mu = 0
sigma = 1
theta_vec = np.linspace(0, np.pi/4, 4)
# create a normal distribution object
n_cols = 2
n_rows = 2
dist        = norm(mu, sigma)
fig, axs    = plt.subplots(ncols=n_cols, nrows=n_rows, figsize=(15,15), subplot_kw={'projection': '3d'})
fig2, axs2  = plt.subplots(ncols=2, nrows=2, figsize=(15,15))
reference_crosses = []
for ax_i, theta in enumerate(theta_vec):
    X, Y, Z, compressed_Z = generate_Zi(pix_i_n, pix_j_n, mu, sigma, bits, theta_vec[ax_i])
    reference_crosses.append(compressed_Z)
    row = ax_i // n_cols
    col = ax_i % n_cols
    axs[row, col].plot_surface(X, Y, Z, cmap='viridis')
    axs[row, col].set_xlabel('X')
    axs[row, col].set_ylabel('Y')
    axs[row, col].set_zlabel('I(x,y)')
    axs[row, col].view_init(elev=90, azim=270) # set azim to 270 degrees
    axs2[row, col].imshow(compressed_Z, cmap='gray')
plt.show()

In [ ]:
pix_set.generate_reference_image(plot=True, seperate_save=True)

In [ ]:
sliders = pix_set.cross_correlate_all()

In [ ]:
# white_value, black_value = pix_set.get_white_black_value()
# pix_set.plot_histogram(bins=50)

# fig, ax = plt.subplots()

# colors = [(1, 0, 0), (0, 1, 0)]
# markers = ['o', 's', 'v', '^', 'D', 'p', '*', 'X', 'd', 'h']
# markersize = 10
# cmap = mcolors.LinearSegmentedColormap.from_list('red_to_green', colors, N=256)

# still_image_temp = still_image.copy() - still_image.mean()
# Low_I = still_image < 20000
# # reference_crosses.append(still_image_temp[263:270, 334:341].astype(np.float64))
# corr_max = np.zeros((still_image.shape[0], still_image.shape[1]))
# for i, reference_cross in enumerate(pix_set.ref_imgs):
#     # template = reference_crosses[0].astype(np.float64)
#     # template -= template.mean()
#     # corr = correlate2d(still_image_temp, template, boundary='symm', mode='same')
#     # corr = match_template(still_image, reference_cross, pad_input=True)
#     # high_corr = corr > 0.75
#     # corr = (corr-np.min(corr)) / np.max(corr-np.min(corr))
#     # peaks_local = detect_peaks(corr)
#     # y, x = np.where(peaks_local & high_corr & Low_I) # & high_corr
#     x, y, corr = pix_set.cross_correlate(reference_cross)
#     ax.scatter(x, y, marker=markers[i % len(markers)], s=markersize*3)
#     markersize -= 1
#     corr_max = np.maximum(corr_max, corr)



ax.imshow(still_image, cmap='gray')
num_plots = len(pix_set.ref_imgs)
if num_plots == 1:
    fig, ax = plt.subplots()
    ax.imshow(pix_set.ref_imgs[0], cmap='gray')
elif num_plots == 2:
    fig, ax = plt.subplots(ncols=2)
    for i, ref_cross in enumerate(pix_set.ref_imgs):
        ax[i].imshow(ref_cross, cmap='gray')
else:
    num_cols = np.ceil(np.sqrt(num_plots))
    num_rows, remainder = divmod(num_plots, num_cols)
    if remainder > 0:
        num_rows += 1

    fig, ax = plt.subplots(nrows = int(num_rows), ncols= int(num_cols))
    for i, ref_cross in enumerate(pix_set.ref_imgs):
        row = int(i // num_cols)
        col = int(i % num_cols)
        ax[row, col].imshow(ref_cross, cmap='gray')
plt.show()


In [ ]:
template = still_image[263:270, 334:341]
result = match_template(still_image, template)
ij = np.unravel_index(np.argmax(result), result.shape)
x, y = ij[::-1]
fig, ax = plt.subplots()
ax.imshow(still_image, cmap='gray')
hcoin, wcoin = template.shape
rect = plt.Rectangle((x, y), wcoin, hcoin, edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.show()

Set limits on the Pixel intensity, Gradient x, and Gradient y to define a Region of Interest (ROI)
By drawing polygons on the figure and pressing enter, parts of the ROI can be removed.

In [ ]:
%matplotlib qt
arg_pi = still_image < 70000
arg_gy = np.linalg.norm([gy, gx], axis=0) > 28000
args = arg_pi & arg_gy #| arg_gx
pix_i, pix_j = np.where(args)


pixelsetter = PixelSetter(still_image, pix_i, pix_j, file_name, sequential_image_n)
pixelsetter.set_method('by_hand')
pixelsetter.get_tracking_pixels()


In [ ]:
pixelsetter.plot_histogram(inside_polygon=True)
pixelsetter.save()

In [ ]:
# del pixelsetter
pixelsetter = PixelSetter.load(file_name)
sequential_image_n = pixelsetter.frame_nr
pix_i = pixelsetter.pix_i
pix_j = pixelsetter.pix_j

Since the above process can be time consuming, the code below can be used to replot the ROI

In [ ]:
# pix_i, pix_j, np.linalg.norm([gy, gx], axis=0).max()

In [ ]:
%matplotlib qt
# mraw_range = (sequential_image_n, sequential_image_n+800) 
# n_frames = mraw_range[1] - mraw_range[0]

# fig, ax = plt.subplots(figsize=(15, 5))
# ax.imshow(still_image, cmap='gray')
# plt.plot(pix_j, pix_i, 'r.', markersize=1)
video.set_points(np.stack((pix_i, pix_j), axis=-1))
video.set_method('lk')
video.method.configure()  #mraw_range=(sequential_image_n, video.N) reference_range=(1,2)
# print((video.method.start_time, video.method.stop_time))
video.show_points()


In [ ]:
# %matplotlib inline
# ref_img = video.method.reference(video.mraw[video.method.reference_range[0]: video.method.reference_range[1]], video.method.subset_size)
# fig, ax = plt.subplots()
# ax.imshow(still_image, cmap='gray')
# plt.show()

In [ ]:
d = video.get_displacements()
# video.method.valid_points
d = video.displacements
d

In [ ]:
%matplotlib qt
fig, ax = plt.subplots()
for point in np.arange(d.shape[0]): #[video.method.valid_points]
    ax.plot(np.arange(video.N), d[point, :, 0], label='point {} - dir 0'.format(point)) #video.info['Record Rate(fps)']*
    ax.plot(np.arange(video.N), d[point, :, 1], label='point {} - dir 1'.format(point)) #video.info['Record Rate(fps)']*
# ax.legend()
# ax.set_ylim(-50, 50)
ax.set_xlabel('frame number')
ax.set_ylabel('displacement [pixels]')
# ax.set_xlim(0, n_frames)

In [ ]:
# Add pix_i and pix_j to displacement to get the tracked position
# mask = np.any(d > 3, axis=(1,2))
tp = (np.stack((pix_i, pix_j), axis=-1)).reshape(len(pix_i),1,2) + d # d[:,:n_frames,:]
# tp[mask, :, :] = np.nan
ani = play_video(video, range(1,799), interval=60, points=tp ) #axis=([720,760],[300,230])axis = ([450,350],[460,560])
# ani = play_video(video, range(10000), interval=30)
writer = animation.FFMpegWriter(fps=60)
path = 'C:/Users/thijs/Documents/GitHub/pyidi_data/anim/'
ani.save(path+file_name + '_LK.mp4', writer=writer)

In [ ]:
import os

folder_path = 'H:/My Drive/PHD/HSC'
all_files_drive = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_drive.append(file)

folder_path = 'D:/HSC'
all_files_T7 = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_T7.append(file)

folder_path = 'F:/'
all_files_Sandisk = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_Sandisk.append(file)


folder_path = 'E:/thijs'
all_files_Passport = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_Passport.append(file)

folder_path = 'C:/Users/thijs/Documents/HSC'
all_files_internal = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_internal.append(file)


In [ ]:
unique_drive_files = set(all_files_drive) - set(all_files_T7) - set(all_files_Sandisk) - set(all_files_Passport) - set(all_files_internal)
unique_drive_files